In [65]:
from ../../hsnf import row_style_hermite_normal_form, column_style_hermite_normal_form

SyntaxError: invalid syntax (<ipython-input-65-c37f9212f8f7>, line 1)

In [33]:
import pandas as pd 
import numpy as np
from scipy.special import comb
import math
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo
import itertools
import random

### 解析用データ作成

（3年間骨肉腫に罹患しない）事象と各特徴量とのP値を計算する。
- Osteogenic sarcoma data:骨肉腫
- LI:リンパ球浸出
- AOP:類骨病理学

In [34]:
ost = pd.read_csv("../input/ost.csv", sep=',', index_col=0)
ost
#論文と照らし合わせ済み

,LI,SEX,AOP,COUNT
Y,,,,
0,0,0,0,0
0,0,0,1,0
0,0,1,0,0
0,0,1,1,0
0,1,0,0,0
0,1,0,1,2
0,1,1,0,4
0,1,1,1,11
1,0,0,0,3


In [35]:
LI_count = 0
for i in range(ost.shape[0]):
    LI_count += ost.iloc[i, 1]*ost.iloc[i, -1]
LI_count    

31

In [36]:
ost46 = pd.DataFrame(np.arange(1*4).reshape(1, 4), columns=['Y', 'LI', 'SEX', 'AOP'])
ost_ = ost.reset_index() #index解除
for i in range(ost_.shape[0]):
    if ost_.iloc[i, -1] > 0:
        for ii in range(ost_.iloc[i, -1]):
            ost46.loc[ost46.shape[0]] = ost_.iloc[i, :-1]

In [37]:
ost46 = ost46.drop(0)

In [38]:
ost46.head()

,Y,LI,SEX,AOP
1,0,1,0,1
2,0,1,0,1
3,0,1,1,0
4,0,1,1,0
5,0,1,1,0


In [39]:
ost46 = ost46.reset_index(drop=True)

In [40]:
ost46.head()

,Y,LI,SEX,AOP
0,0,1,0,1
1,0,1,0,1
2,0,1,1,0
3,0,1,1,0
4,0,1,1,0


<img src = '../input/about_SA.png'>

- [SA](https://qiita.com/ShataKurashi/items/c0c6044e97fa9e4a9471#:~:text=%E7%84%BC%E3%81%8D%E3%81%AA%E3%81%BE%E3%81%97%E6%B3%95%EF%BC%88%E3%82%84%E3%81%8D%E3%81%AA%E3%81%BE%E3%81%97,%E3%81%A6%E3%80%81%E3%82%88%E3%81%84%E8%BF%91%E4%BC%BC%E3%82%92%E4%B8%8E%E3%81%88%E3%82%8B%E3%80%82)
- [組み合わせ最適問題に対するSA法　中野秀男・中西義郎](http://www.orsj.or.jp/~archive/pdf/bul/Vol.31_01_043.pdf)
- [4ti2 マルコフ基底](https://github.com/4ti2/4ti2)
- [desr](https://desr.readthedocs.io/en/latest/hermite_smith_doc.html)
  - [desr github](https://github.com/tanbur/desr):python2のようだ

In [45]:
def each_energy(y_sample, t1): #毎回のmaskとyが引数、合致度合いの計算を行う
    energy_except_LI = (np.dot(ost46['SEX'], y_sample)-16)**2 + (np.dot(ost46['AOP'], y_sample) - 12)**2 + (sum(y_sample) - 29)**2
    energy_LI = (np.dot(ost46['LI'], y_sample) - t1)**2
    return energy_except_LI + energy_LI

In [62]:
y_list_num = {}
y_list_variation = {}
each_energy_list = []
t_init = 1e4 #10000
y_sample = ost46['Y'].copy()


for t1 in range(19, 30):
    y_list_num[t1] = 0
    y_list_variation[t1] = []
    for t in reversed(range(1, int(t_init))):
        i = random.randrange(ost46.shape[0])
        y_sample[i] = 1 - y_sample[i]
        if each_energy(y_sample, t1) == 0:
            y_list_num[t1] += 1
            y_list_variation[t1].append(y_sample)
            each_energy_list.append(0)
        elif len(each_energy_list) > 0:
            if np.random.rand() >= np.exp((each_energy_list[-1]-each_energy(y_sample, t1))/t):
                y_sample[i] = 1 - y_sample[i]

In [63]:
y_list_num

{19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0}

#### 論文記載のTable1を再現

In [ ]:
ost_original_LISEXAOP = [
    [0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]
]
len(ost_original_LISEXAOP)

In [ ]:
ost_original = pd.DataFrame(np.arange(32).reshape(8, 4), columns=['LI', 'SEX', 'AOP', 'Propotion DFI3'])

pro_pra = [0]*8
pro_mina = [0]*8
for i in range(ost.shape[0]):
    for ii in range(len(ost_original_LISEXAOP)):
        if list(ost.iloc[i, 0:3])==ost_original_LISEXAOP[ii]:
            if ost.index[i]==1:
                pro_pra[ii] += ost.iloc[i, 3]
            else:
                pro_mina[ii] += ost.iloc[i, 3]

print(pro_pra)
print(pro_mina)  

In [ ]:
for i in range(len(ost_original_LISEXAOP)):#num_samples
    ost_original.iloc[i, :3] = ost_original_LISEXAOP[i]
    ost_original.iloc[i, 3] = round([pro_pra[k]/(pro_pra[k]+pro_mina[k]) for k in range(len(pro_pra))][i], 2)
ost_original

In [ ]:
ost.head()

### (t_0, t_1, t_2, t_3)を表示
- t_i = 「（3年後に骨肉腫に罹患していない(=1)）かつ（特徴量iが1である）」サンプル数

In [ ]:
t_list = []
count_y = sum([ost.index[i]*ost.iat[i, 3] for i in range(ost.shape[0])])#単に、3年後に骨肉腫に罹患していないサンプル数
t_list.append(count_y)
for j in range(3):
    count = sum([ost.iat[i, j]*ost.iat[i, 3] for i in range(ost.shape[0]) if ost.index[i]==1])
    t_list.append(count)

In [ ]:
t_list#Y, LI, SEX, AOP

### t_1(LI)以外固定してサンプリング

In [ ]:
pacient_num = sum(ost.iloc[:, -1])

In [ ]:
li = Array.create('LI', shape=pacient_num, vartype='BINARY')
H = (sum([li[i]*ost.iloc[i, -1] for i in range(ost.shape[0])]) - t_list[1])**2 #19になるように固定した
qb = H.compile().to_qubo()
sa_sampler = SimulatedAnnealingSampler()

res = sa_sampler.sample_qubo(qb[0], num_reads=100) 

In [ ]:
print(len(res))

In [ ]:
for li_info in list(res.record):
    print(li_info[0])

In [ ]:
y = Array.create('Y', shape=pacient_num, vartype='BINARY')
sex = Array.create('SEX', shape=pacient_num, vartype='BINARY')
aop = Array.create('AOP', shape=pacient_num, vartype='BINARY')
li = Array.create('LI', shape=pacient_num, vartype='BINARY')

H = (sum(y) - t_list[0])**2 + (sum(sex) - t_list[2])**2 + (sum(aop) - t_list[3])**2
qb = H.compile().to_qubo()
sa_sampler = SimulatedAnnealingSampler()

res = sa_sampler.sample_qubo(qb[0], num_reads=100)

In [ ]:
res.record

In [ ]:
ost

In [ ]:
#超幾何分布の見本
feature_Y = pd.DataFrame(columns=['feature1', 'feature0', 'feature_sum'],index=['Y1', 'Y0', 'Y_sum'])
feature_Y.iloc[0, 0] = 'k'
feature_Y.iloc[0, 2] = 'K'
feature_Y.iloc[2, 0] = 'n'
feature_Y.iloc[2, 2] = 'N'

feature_Y

In [ ]:
LI_Y = pd.DataFrame(columns=['LI1', 'LI0', 'LIsum'],index=['Y1', 'Y0', 'Ysum'])
LI_Y.fillna(0, inplace=True)
LI_Y

In [ ]:
for i in range(ost.shape[0]):
    if ost.index[i] == 1 and ost.iloc[i, 0] == 1:
        LI_Y.iloc[0, 0] += ost.iloc[i, -1]
    elif ost.index[i] == 1 and ost.iloc[i, 0] == 0:
        LI_Y.iloc[0, 1] += ost.iloc[i, -1]
    elif ost.index[i] == 0 and ost.iloc[i, 0] == 1:
        LI_Y.iloc[1, 0] += ost.iloc[i, -1]
    else:
        LI_Y.iloc[1, 1] += ost.iloc[i, -1]

#print(LI_Y)
LI_Y.iloc[0, 2] = LI_Y.iloc[0, 0] + LI_Y.iloc[0, 1]
LI_Y.iloc[1, 2] = LI_Y.iloc[1, 0] + LI_Y.iloc[1, 1]
LI_Y.iloc[2, 0] = LI_Y.iloc[0, 0] + LI_Y.iloc[1, 0]
LI_Y.iloc[2, 1] = LI_Y.iloc[0, 1] + LI_Y.iloc[1, 1]

if LI_Y.iloc[0, 2] + LI_Y.iloc[1, 2] == LI_Y.iloc[2, 0] + LI_Y.iloc[2, 1]:
    LI_Y.iloc[2, 2] = LI_Y.iloc[2, 0] + LI_Y.iloc[2, 1]
else:
    print('error is occured')

In [ ]:
LI_Y

In [ ]:
LI_Y_list = LI_Y.values.tolist()
LI_Y_list

超幾何分布を考えると、$K=29, n=36, k=19, N=46$

マルコフ基底において固定する十分統計量は$29, 17, 46, 10, 36$（周辺和）

In [ ]:
#NCn
print(comb(46, 36, exact=True))
print(comb(LI_Y.iloc[2, 2], LI_Y.iloc[2, 0], exact=True))

In [ ]:
KCk = comb(LI_Y.iloc[0, 2], LI_Y.iloc[0, 0], exact=True)
NKCnk = comb(LI_Y.iloc[1, 2], LI_Y.iloc[1, 0], exact=True)
NCn = comb(LI_Y.iloc[2, 2], LI_Y.iloc[2, 0], exact=True)
p_ori = KCk*NKCnk/NCn
p_ori

In [ ]:
p_ori_chi = math.factorial(LI_Y.iloc[0, 2])*math.factorial(LI_Y.iloc[1, 2])*math.factorial(LI_Y.iloc[2, 1])*math.factorial(LI_Y.iloc[2, 0])
p_ori_mom = math.factorial(LI_Y.iloc[2, 2])*math.factorial(LI_Y.iloc[0, 0])*math.factorial(LI_Y.iloc[0, 1])*math.factorial(LI_Y.iloc[1, 0])*math.factorial(LI_Y.iloc[1, 1])
p_ori = p_ori_chi/p_ori_mom
p_ori

In [ ]:
p_ori_chi = math.factorial(LI_Y_list[0][-1])*math.factorial(LI_Y_list[1][-1])*math.factorial(LI_Y_list[2][1])*math.factorial(LI_Y_list[2][0])
p_ori_mom = math.factorial(LI_Y_list[2][-1])*math.factorial(LI_Y_list[0][0])*math.factorial(LI_Y_list[0][1])*math.factorial(LI_Y_list[1][0])*math.factorial(LI_Y_list[1][1])
p_ori = p_ori_chi/p_ori_mom
p_ori


In [ ]:
p_sum = 0
range_num = min(LI_Y.iloc[0, 2], LI_Y.iloc[2, 0])+1
for k in range(0, range_num): #k = LI_Y.iloc[0, 0]として
    LI_Y.iloc[0, 0] = k
    LI_Y.iloc[0, 1] = LI_Y.iloc[0, 2] - k
    LI_Y.iloc[1, 0] = LI_Y.iloc[2, 0] - k
    LI_Y.iloc[1, 1] = LI_Y.iloc[2, 2] - LI_Y.iloc[0, 2] - (LI_Y.iloc[2, 0] - k)
    if min(k, LI_Y.iloc[0, 1], LI_Y.iloc[1, 0], LI_Y.iloc[1, 1]) >= 0:
        print(LI_Y)
        KCk = comb(LI_Y.iloc[0, 2], k, exact=True)
        NKCnk = comb(LI_Y.iloc[1, 2], LI_Y.iloc[1, 0], exact=True)
        NCn = comb(LI_Y.iloc[2, 2], LI_Y.iloc[2, 0], exact=True)
        p = KCk*NKCnk/NCn
        print('p =',p)
        
        p_sum += p
        print('p_sum =', p_sum)

p_sum